## ANALYST

In [2]:
import pandas as pd


df = pd.read_csv("../data/processed/bank_campaign_synthetic_200.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          200 non-null    int64 
 1   job          200 non-null    object
 2   marital      200 non-null    object
 3   education    200 non-null    object
 4   default      200 non-null    object
 5   balance      200 non-null    int64 
 6   housing      200 non-null    object
 7   loan         200 non-null    object
 8   contact      200 non-null    object
 9   day_of_week  200 non-null    int64 
 10  month        200 non-null    object
 11  duration     200 non-null    int64 
 12  campaign     200 non-null    int64 
 13  pdays        200 non-null    int64 
 14  previous     200 non-null    int64 
 15  poutcome     200 non-null    object
 16  y            200 non-null    object
dtypes: int64(7), object(10)
memory usage: 26.7+ KB


In [4]:
df = df.drop(columns=['y'])

In [5]:
df['age_group'] = pd.cut(
    df['age'],
    bins=[0, 29, 58, 120],
    labels=['<29', '29–58', '>58']
)

df['balance_group'] = pd.cut(
    df['balance'],
    bins=[-1e9, 60.5, 798.5, 1578.5, 1e9],
    labels=['low', 'medium', 'high', 'very_high']
)


In [6]:
best_mask = (
    (df['housing'] == 'no') &
    (df['age_group'] == '>58') &
    (df['balance_group'].isin(['high', 'very_high']))
)


In [7]:
potential_mask = (
    (df['housing'] == 'no') &
    (df['age_group'] == '<29') &
    (df['balance_group'].isin(['medium', 'high', 'very_high']))
)


In [8]:
inhibiting_mask = (
    (df['housing'] == 'yes') &
    (df['age_group'] == '29–58') &
    (df['balance_group'] == 'low')
)


In [9]:
df['customer_tier'] = 'Neutral'

df.loc[best_mask, 'customer_tier'] = 'Best'
df.loc[potential_mask, 'customer_tier'] = 'Potential'
df.loc[inhibiting_mask, 'customer_tier'] = 'Inhibiting'


In [10]:
df['campaign_action'] = 'Review'

df.loc[df['customer_tier'] == 'Best', 'campaign_action'] = 'Call Immediately'
df.loc[df['customer_tier'] == 'Potential', 'campaign_action'] = 'Nurture Campaign'
df.loc[df['customer_tier'] == 'Inhibiting', 'campaign_action'] = 'Exclude from Deposit Campaign'


In [11]:
df.loc[df['poutcome'] == 'success', 'campaign_action'] = 'High Priority Call'

df.loc[
    (df['poutcome'] == 'failure') &
    (df['customer_tier'] != 'Best'),
    'campaign_action'
] = 'Delay & Re-nurture'


In [12]:
df['customer_tier'].value_counts(normalize=True)


customer_tier
Neutral      0.79
Best         0.12
Potential    0.09
Name: proportion, dtype: float64

In [14]:
df.to_csv("../data/processed/campaign_execution_plan.csv", index=False)
